# Undersampling

Implement the cluster centroids and random undersampling techniques with the credit card default data. Then estimate a logistic regression model and report the classification evaluation metrics from both sampling methods. 

ln_balance_limit is the log of the maximum balance they can have on the card; 1 is female, 0 male for sex; the education is denoted: 1 = graduate school; 2 = university; 3 = high school; 4 = others; 1 is married and 0 single for marriage; default_next_month is whether the person defaults in the following month (1 yes, 0 no).

In [1]:
import pandas as pd
from path import Path
from collections import Counter

In [2]:
data = Path('../Resources/cc_default.csv')
df = pd.read_csv(data)
df.head()

ID  ln_balance_limit  sex  education  marriage  age  default_next_month
0   1          9.903488    1          2         0   24                   1
1   2         11.695247    1          2         1   26                   1
2   3         11.407565    1          2         1   34                   0
3   4         10.819778    1          2         0   37                   0
4   5         10.819778    0          2         0   57                   0

In [6]:
x_cols = [i for i in df.columns if i not in ('ID', 'default_next_month')]
X = df[x_cols]
y = df['default_next_month']


In [9]:
# Normal train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train

ln_balance_limit  sex  education  marriage  age
26340         11.918391    1          1         1   26
3877          10.819778    1          2         1   23
27081         11.289782    0          2         1   31
5440          11.695247    1          2         1   39
22164         11.918391    1          1         0   30
...                 ...  ...        ...       ...  ...
17289         12.043554    1          3         0   40
5192          12.706848    1          1         0   41
12172         10.819778    0          2         1   22
235           11.289782    0          1         1   35
29733         12.611538    0          2         0   37

[22500 rows x 5 columns]

## Random Undersampling

In [10]:
# Undersample the data using `RandomUnderSampler`
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 4968, 1: 4968})

In [11]:
# Fit a Logistic regression model using random undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[3732, 2100],
       [ 740,  928]], dtype=int64)

In [13]:
# Calculate the Balanced Accuracy Score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5981363057701987

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.64      0.56      0.72      0.60      0.36      5832
          1       0.31      0.56      0.64      0.40      0.60      0.35      1668

avg / total       0.72      0.62      0.57      0.65      0.60      0.36      7500



## Cluster Centroid Undersampling

In [15]:
# Fit the data using `ClusterCentroids` and check the count of each class
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 4968, 1: 4968})

In [16]:
# Logistic regression using cluster centroid undersampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[2811, 3021],
       [ 601, 1067]], dtype=int64)

In [20]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5608420670870711

In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.82      0.48      0.64      0.61      0.56      0.30      5832
          1       0.26      0.64      0.48      0.37      0.56      0.31      1668

avg / total       0.70      0.52      0.60      0.56      0.56      0.31      7500

